# Introduction
The previous stage includes reviews text preprocessing, which is in Process_Reviews.ipynb<br>
The following stage is aim to analysis reviews sentiment

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('punkt')
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
import seaborn as sns
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from gensim.models import Word2Vec
import spacy
nlp = spacy.load('en_core_web_sm')
import string
import sys
import os
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
    os.environ["PYTHONWARNINGS"] = "ignore" 

import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\AA\AppData\Roaming\nltk_data...
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AA\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\AA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load reviews

In [2]:
file = "Cleaned_Text_Dataset.csv"
df = pd.read_csv(file)
del df["Unnamed: 0"]
df.head()

,review_id,business_id,text,target,words
0,Gt_h5A-A5Nt50EbaPCorvw,F8yozE3NWnImNApHO347gQ,First impression was good but their food is ho...,0.0,impression good food horrible cash idea pricey
1,0dELc-isYD5Av7KNsIvcRA,WxB8498ejPtHE7wFa89_fA,"I've been to this location countless times, an...",0.0,location countless time time food service ambi...
2,Bmrq4dvmlVMGU3roXzeDgQ,-mIlmp5l4hKlp1tvHRdvTg,Seems popular- but not that delish. Short sub ...,0.0,popular delish short sub maybe typical length ...
3,tCP2EjtzGJ7JEHlj_8i1xw,hcxea89M_U__LADtu3C0kA,The service was great! The place was very beau...,0.0,service great place beautiful small tight rest...
4,3gM_kcsqfU9eqmE19kL4tw,3BJxm-HnvzdwD1zjmSbmyQ,I was at this restaurant recently and was so u...,0.0,restaurant recently unhappy purchase wonton so...


# Sentiment polarity

我们将情绪分为四个类别，negative、neutral、positive and compound。【摘抄】The first three are easy to understand and for the compound score, it is a combination of positive and negative scores and ranges from -1 to 1: below 0 is negative and above 0 is positive. I am going to use the compound score to measure the sentiment.前三个很容易理解，对于复合分数，它是正分数和负分数的组合，范围从 -1 到 1：低于 0 为负，高于 0 为正。我将使用复合分数来衡量情绪。

In [3]:
# Instantiate new SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
# Generate sentiment scores
sentiment_scores = df['words'].apply(sid.polarity_scores)
sentiment = sentiment_scores.apply(lambda x: x['compound'])
print(df['words'][0])
print(sentiment_scores.head())

impression good food horrible cash idea pricey
0    {'neg': 0.285, 'neu': 0.325, 'pos': 0.39, 'com...
1    {'neg': 0.034, 'neu': 0.694, 'pos': 0.271, 'co...
2    {'neg': 0.059, 'neu': 0.46, 'pos': 0.481, 'com...
3    {'neg': 0.21, 'neu': 0.405, 'pos': 0.385, 'com...
4    {'neg': 0.169, 'neu': 0.637, 'pos': 0.195, 'co...
Name: words, dtype: object


# Split training set and testing set

In [4]:
X = df['words'] 
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42, stratify=y)
print("X_train:\n", X_train.head())
print("\ny_train:\n", y_train.head())

X_train:
 4882    similar reviewer avoid place year simply veget...
4931    happy pop major change work long table got rid...
275     month track burger fi like iroquois brave prov...
6592    husband dine wedding anniversary note occasion...
7150    love reading terminal shop fish head pick hoag...
Name: words, dtype: object

y_train:
 4882    1.0
4931    1.0
275     0.0
6592    1.0
7150    1.0
Name: target, dtype: float64


# Cross-Validation

列举我们将要实验的n-gram，【摘抄】GridSearchCV是Sklearn model_selection包的一个模块，用于超参数调整。 给定一组不同的超参数，GridSearchCV 循环浏览所有可能的超参数值和组合，并在训练数据集上拟合模型。 在这个过程中，它能够确定产生最佳精度的超参数的最佳值和组合（从给定的参数集中）【摘抄】在机器学习模型中，需要人工选择的参数称为超参数。比如随机森林中决策树的个数，人工神经网络模型中隐藏层层数和每层的节点个数，正则项中常数大小等等，他们都需要事先指定。超参数选择不恰当，就会出现欠拟合或者过拟合的问题。而在选择超参数的时候，有两个途径，一个是凭经验微调，另一个就是选择不同大小的参数，带入模型中，挑选表现最好的参数。微调的一种方法是手工调制超参数，直到找到一个好的超参数组合，这么做的话会非常冗长，你也可能没有时间探索多种组合，所以可以使用Scikit-Learn的GridSearchCV来做这项搜索工作。<br>
这里用的是后者<br>
可以提一下交叉验证，cross validation

In [5]:
param_grid = {'c_vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3), (2, 2), (2, 3), (3, 3)]}

### Bag-of-words model(wordcounts) and Vectorisation

### Logistic Regression model 
找出哪个n-gram在逻辑回归模型中表现更好

In [6]:
lr_pipeline = Pipeline([
    ('c_vectorizer', CountVectorizer()),
    ('lr', LogisticRegression(random_state=42))
])

gs_lr = GridSearchCV(lr_pipeline, refit=True, cv=2, param_grid=param_grid, scoring='f1', n_jobs=-1)
gs_lr.fit(X_train, y_train)

print('optimal n-gram: ', gs_lr.best_estimator_.get_params()['c_vectorizer__ngram_range'])
print("optimal parameter: ", gs_lr.best_params_)
print("optimal score: ", gs_lr.best_score_)

print('classification report')
predictions = gs_lr.predict(X_test)
report = classification_report(y_test, predictions, digits=4, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report

optimal n-gram:  (1, 2)
optimal parameter:  {'c_vectorizer__ngram_range': (1, 2)}
optimal score:  0.9555176337108469
classification report


,precision,recall,f1-score,support
0.0,0.958333,0.993566,0.975632,1088.000000
1.0,0.993321,0.956801,0.974719,1088.000000
accuracy,0.975184,0.975184,0.975184,0.975184
macro avg,0.975827,0.975184,0.975175,2176.000000
weighted avg,0.975827,0.975184,0.975175,2176.000000


### Support Vector Machine model 
找出哪个n-gram在支持向量机SVM模型中表现更好

In [7]:
svm_pipe = Pipeline([
    ('c_vectorizer', CountVectorizer()),
    ('svm', svm.SVC(max_iter=-1, random_state=42))
])

gs_svm = GridSearchCV(svm_pipe, refit=True, cv=2, param_grid=param_grid, scoring='f1', n_jobs=-1)
gs_svm.fit(X_train, y_train)

print('optimal n-gram: ', gs_svm.best_estimator_.get_params()['c_vectorizer__ngram_range'])
print("optimal parameter: ", gs_svm.best_params_)
print("optimal score: ", gs_svm.best_score_)

print('classification report')
predictions = gs_svm.predict(X_test)
report = classification_report(y_test, predictions, digits=4, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report

optimal n-gram:  (1, 2)
optimal parameter:  {'c_vectorizer__ngram_range': (1, 2)}
optimal score:  0.9467008914841224
classification report


,precision,recall,f1-score,support
0.0,0.964448,0.972426,0.968421,1088.00000
1.0,0.972196,0.964154,0.968159,1088.00000
accuracy,0.968290,0.968290,0.968290,0.96829
macro avg,0.968322,0.968290,0.968290,2176.00000
weighted avg,0.968322,0.968290,0.968290,2176.00000


【这里要改】对比上面逻辑回归和SVM交叉验证的结果，逻辑回归的最佳性能更好，因此我们选择最佳性能更好的逻辑回归结果，它的最优参数n-gram是（1，2）

# Supervised Learning Classifiers

In [8]:
print("X_train:\n", X_train.head())
print("\ny_train:\n", y_train.head())

X_train:
 4882    similar reviewer avoid place year simply veget...
4931    happy pop major change work long table got rid...
275     month track burger fi like iroquois brave prov...
6592    husband dine wedding anniversary note occasion...
7150    love reading terminal shop fish head pick hoag...
Name: words, dtype: object

y_train:
 4882    1.0
4931    1.0
275     0.0
6592    1.0
7150    1.0
Name: target, dtype: float64


### Bag-of-words model(TF-IDF) and Vectorisation
【摘抄】we can use TF_IDF vectorizing to find the weighted words that occur more frequently in the document that leads to creation of the bag of words model我们可以使用 TF_IDF 向量化来找到文档中出现频率更高的加权词，从而创建词袋模型

In [9]:
# Stopwords
my_stop_words = set(stopwords.words('english') + 
                    list(ENGLISH_STOP_WORDS) + 
                    ['super', 'duper', 'place'])
exclude_stopwords = ['no','not','none']
for word in exclude_stopwords:
    my_stop_words.remove(word)

In [10]:
def my_tokenizer(sentence):
    # to remove any space from beginning and the end of text
    listofwords = sentence.strip().split()
    listof_words = []    
    for word in listofwords:
        if not word in my_stop_words:
            lemm_word = WordNetLemmatizer().lemmatize(word)
            # remove the stop words
            for punctuation_mark in string.punctuation:
                word = word.replace(punctuation_mark, '').lower()
            if len(word)>0:
                listof_words.append(word)
    return listof_words

从上面cross-validation得到最优的n-gram的结果(1,2)，在这里使用

In [11]:
vect_1 = TfidfVectorizer(min_df=100,
                         tokenizer=my_tokenizer,
                         stop_words=list(my_stop_words), 
                         ngram_range=(1,2)).fit(X_train)
X_train1 = vect_1.transform(X_train)
X_test1 = vect_1.transform(X_test)
# the below line for future coeff
X_train1_df = pd.DataFrame(X_train1.toarray(), columns=vect_1.get_feature_names_out())

In [12]:
new_df_words = pd.DataFrame(columns=vect_1.get_feature_names_out(), data=X_train1.toarray())
new_df_words

,able,absolutely,actually,add,ago,amazing,ambiance,appetizer,area,arrive,...,woman,wonderful,work,worth,wow,write,wrong,year,yelp,yes
0,0.0,0.0,0.0,0.0,0.215433,0.150120,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.158759,0.0,0.0,0.000000,0.172716,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.223475,0.0,...,0.0,0.0,0.211848,0.000000,0.0,0.0,0.000000,0.233372,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.054746,0.000000,0.0,0.0,0.064513,0.000000,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.112143,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.268046
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5071,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
5072,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
5073,0.0,0.0,0.0,0.0,0.000000,0.137246,0.0,0.000000,0.151208,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000
5074,0.0,0.0,0.0,0.0,0.000000,0.159428,0.0,0.185247,0.000000,0.0,...,0.0,0.0,0.000000,0.168603,0.0,0.0,0.000000,0.000000,0.0,0.000000


【可以转csv，用作推荐】

In [13]:
#counting most repetitive words 
word_counts = np.array(np.sum(X_train1, axis=0)).reshape((-1,))
words = np.array(vect_1.get_feature_names_out())
words_df = pd.DataFrame({"words":words, "counts":word_counts})
words_df.sort_values(by="counts",ascending=False).head(15)

,words,counts
150,food,307.563976
164,good,284.709831
280,order,222.567556
170,great,218.182862
417,time,189.943344
72,come,188.883089
223,like,188.783906
362,service,182.852285
335,restaurant,160.727427
428,try,148.129300


### Logistic Regression model

In [14]:
# Fitting Logistic regression to the training set
logreg = LogisticRegression(solver='lbfgs',multi_class='auto',random_state=1)
logreg.fit(X_train1, y_train)

# Predicting the test set results
y_pred_logreg = logreg.predict(X_test1)

# Training score
print(f"Score on training set: {logreg.score(X_train1,y_train)}")
print(f"Score on test set: {logreg.score(X_test1,y_test)}")

Score on training set: 0.9456264775413712
Score on test set: 0.9223345588235294


In [15]:
print('confusion matrix')
con_mat_lr = confusion_matrix(y_test, y_pred_logreg)
df_cm_lr = pd.DataFrame(con_mat_lr, columns = ['Predicted 0','Predicted 1'], index = ['True 0','True 1'])
display(df_cm_lr)
print('classification report')
report = classification_report(y_test, y_pred_logreg, output_dict=True)
df_report = pd.DataFrame(report).transpose()
df_report

confusion matrix


,Predicted 0,Predicted 1
True 0,1019,69
True 1,100,988


classification report


,precision,recall,f1-score,support
0.0,0.910634,0.936581,0.923425,1088.000000
1.0,0.934721,0.908088,0.921212,1088.000000
accuracy,0.922335,0.922335,0.922335,0.922335
macro avg,0.922678,0.922335,0.922319,2176.000000
weighted avg,0.922678,0.922335,0.922319,2176.000000


In [16]:
# find the most informative words
log_odds = logreg.coef_[0]
coeff = pd.DataFrame(log_odds, X_train1_df.columns, columns=['coef']).sort_values(by='coef', ascending=False)
coeff

,coef
delicious,5.218912
great,4.798466
amazing,4.687341
good,4.410005
definitely,4.171167
...,...
mediocre,-2.979704
disappointed,-3.333566
terrible,-3.609629
bad,-3.740723


### Naive Bayes model

# Testing Sentiment Classifer

In [17]:
def remove_special(text):
    # remove the URL
    text = re.sub(r"http\S+", "", text)
    # remove mentions
    text = re.sub("@[^\s]*", "", text)
    # remove hashtags
    text = re.sub("#[^\s]*", "", text)
    
    return text

nlp = spacy.load('en_core_web_sm')
def spacy_process(text):
    doc = nlp(text)
    # Lemmatization with Spacy
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)
            
    #Filter the stopwords, remove non-letters and lower case 
    #filtered_sentence =[]
    #for word in lemma_list:
        #lexeme = nlp.vocab[word]
        #if lexeme.is_stop == False:
            #filtered_sentence.append(word)
    lower_words = []
    for word in lemma_list:
        filtered_aplha_char = re.sub("[^\w]" , " " , word)
        filtered_single_n = re.sub("n\sn", " " ,filtered_aplha_char)
        text_letters_only = re.sub("[^a-zA-Z]", " ", filtered_single_n)
        text_words_lower = text_letters_only.lower()
        remove_single_char = re.sub(r'\b\w\b', '', text_words_lower)
        lower_words.append(remove_single_char)
    text_final = " ".join(lower_words)
    return  " ".join(text_final.split())


def remove_stopwords(text):
    # Stopwords
    my_stop_words = set(stopwords.words('english') + 
                        list(ENGLISH_STOP_WORDS) + 
                        ['super', 'duper', 'place'])
    exclude_stopwords = ['no','not','none']
    for word in exclude_stopwords:
        my_stop_words.remove(word)
    
    word_tokens = word_tokenize(text)
    tokens_list = list()
    for word in word_tokens:
        if word.isalpha() and word not in my_stop_words:
            tokens_list.append(word)
    return tokens_list

def clean_data(text, needed_format):
    text = remove_special(text)
    words_sentence = spacy_process(text)
    tokens_list = remove_stopwords(words_sentence)
    if needed_format == 'list':
        return tokens_list
    elif needed_format == 'string':
        return words_sentence

In [18]:
def testing_model(text):
    print("Original review is:\n", text)
    print("\nCleaned review is:\n", clean_data(text, 'string'))
    result = logreg.predict(vect_1.transform([clean_data(text,'string')]))
    print("\nLogistic Regression model: ", result)
    if result == 0:
        print("\nThis review has negetive sentiment\n")
    elif result == 1:
        print("\nThis review has positive sentiment\n")

In [22]:
text = "i like the pork burger, and the pizza is delicious, but the wait time is too long"
testing_model(text)

Original review is:
 i like the pork burger, and the pizza is delicious, but the wait time is too long

Cleaned review is:
 like the pork burger and the pizza be delicious but the wait time be too long

Logistic Regression model:  [1.]

This review has positive sentiment



In [179]:
text = " people that works here for sure is friendly! :)I do love that big menu book and seems like there are a lot of items to choose from. This is always nice as Vietnamese food is definitely more than pho and more spring rolls."
testing_model(text)

Original review is:
  people that works here for sure is friendly! :)I do love that big menu book and seems like there are a lot of items to choose from. This is always nice as Vietnamese food is definitely more than pho and more spring rolls.

Cleaned review is:
 people that work here for sure be friendly do love that big menu book and seem like there be lot of item to choose from this be always nice as vietnamese food be definitely more than pho and more spring roll

Logistic Regression model:  [1.]

This review has positive sentiment

